RNN 실습

In [126]:
# imdb 로 data 생성

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# data 가져오기
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)       

# index 수정
word_index = imdb.get_word_index()                                           # word의 index 가져오기
#print(word_index["apple"])                                
word_index = {key: value+3 for key, value in word_index.items()}            # 기본값 추가
#print(word_index["apple"])                      
word_index['<PAD>'] = 0      #기본값                             
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3
reverse_word_index = {value: key for key, value in word_index.items()}       # reverse index
for i in range(100,110): print(reverse_word_index[i])

# data 확인
print('\n')
for i in range(0,10) : print(decode_review(X_train[i]))

# 최대길이 512로 문단 줄이기
X_train = pad_sequences(X_train, value=0, padding='post', maxlen=512)
X_test = pad_sequences(X_test, value=0, padding='post', maxlen=512)
for i in range(0,10) : print(len(X_train[i]))

could
any
movies
after
think
characters
watch
two
films
character


<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the who

512
512
512
512
512
512
512
512
512
512


In [40]:
# 문장 만드는 함수

def decode_review(vector):
    return ' '.join([reverse_word_index[key] for key in vector])

In [182]:
# DNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

optimizer = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #lr:learining rate을 키움 기본은 0.001?

model_DNN = Sequential()
#model_DNN.add(Dense(512, activation='relu', input_shape=(512,)))   #input
model_DNN.add(Embedding(num_words, 16, input_shape=(512,)))
model_DNN.add(Flatten(input_shape=(512,16)))

In [183]:
model_DNN.add(Dense(512, activation='relu', input_shape=(512,) ))
model_DNN.add(Dense(1, activation='sigmoid')) #output

model_DNN.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
model_DNN.fit(X_train, y_train, epochs=1, verbose=1, batch_size=512)

49/49 [==============================] - 5s 95ms/step - loss: 11.5877 - acc: 0.4937


In [180]:
model_DNN.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, None, 16)          160064    
_________________________________________________________________
flatten_13 (Flatten)         (None, None)              0         
Total params: 160,064
Trainable params: 160,064
Non-trainable params: 0
_________________________________________________________________


In [129]:
# model_DNN 확인

model=model_DNN

(test_loss, test_acc) = model.evaluate(X_test,y_test)
print('{:.2f}%\n'.format(100*test_acc))

#print(model.predict(X_test[0]))

model.layers

782/782 [==============================] - 1s 879us/step - loss: 0.6930 - acc: 0.5000
50.00%



In [164]:
# RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM, Flatten

num_words = 10004
optimizer = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #lr:learining rate을 키움 기본은 0.001?

# LSTM 
model_rnn = Sequential()
model_rnn.add(Embedding(num_words, 16, input_shape=(None,)))
#model_rnn.add(SimpleRNN(16))
model_rnn.add(LSTM(16))
#model_rnn.add(Flatten(input_shape=(512,16)))
model_rnn.add(Dense(1,activation='sigmoid'))

model_rnn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
model_rnn.fit(X_train, y_train, epochs=15, verbose=1, batch_size=512) # batch size를 크게함 기본은 30?

49/49 [==============================] - 26s 523ms/step - loss: 0.6951 - acc: 0.5010


In [165]:
# model_rnn 확인

import numpy as np

model=model_rnn

(test_loss, test_acc) = model.evaluate(X_test,y_test)
print('{:.2f}%\n'.format(100*test_acc))

print(model_rnn.predict(np.array(X_test[0]).reshape(1,-1)))

model.layers

782/782 [==============================] - 18s 24ms/step - loss: 0.6917 - acc: 0.5060
50.60%

[[0.49702215]]


In [160]:
# 나의 리뷰 넣어서 해보기

mystring = "this movie is great"
mylist = mystring.split(' ')            # 단어별로 찢기
test_vec = encode_review(mylist)        # word index 로 표시
test_vec = [1]+test_vec 
model_rnn.predict(np.array(test_vec).reshape(1,-1))

array([[0.93564415]], dtype=float32)

In [143]:
def encode_review (word_list):
    return [word_index[word] for word in word_list]